# Generates watchlist for IBKR to import into

In [14]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "SNP"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd 
from loguru import logger 

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util # type: ignore

util.startLoop()

logger.add(sink=ROOT / "log" / "snp.log", mode="w")

3

In [32]:

from snp import quick_pf, get_ib, get_pickle, get_open_orders

# Get positions and orders to remove
with get_ib('SNP') as ib:
    positions = quick_pf(ib)
    openords = get_open_orders(ib).symbol.to_list()
    pos_symbols = positions.symbol.to_list()
    remove = set(pos_symbols).union(((openords)))

# Get clean targets from unds
df_unds = get_pickle(ROOT / 'data' / 'snp_unds.pkl')
df_unds = df_unds[~df_unds.ib_symbol.isin(remove)]
df_unds = df_unds.sort_values('ib_symbol').reset_index(drop=True)

# Write to CSV
csv_txt = [f"DES:{c.symbol}"+":STK:SMART/AMEX,,,,," for c in df_unds.contract]
with open(ROOT / 'data' / 'watchlist.csv', 'w', newline='') as file:
    for item in csv_txt:
        file.write(item.replace(':', ',') + '\n')